# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# import libraries

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline


In [3]:
import os
os.path.abspath(os.getcwd())

'/home/workspace'

### Loading Up DATABASE 'disaster_response' Prepared from ETL Stage

In [4]:
# load data from database
#def load_data(data_file)
def load_data():
    engine = create_engine('sqlite:///disaster_response.db')
    conn = engine.connect()
    df = pd.read_sql_table('disaster_response', conn)
    #df.head(2)
    X = df.message.values
    
    y = df.iloc[:, 4:]
    ##y.dropna(axis = 0, how = 'any', inplace=True)
    ##y.fillna(0, inplace=True)
    colnames = y.columns
    #y = y.values
    engine.dispose()
    
    return X, y, colnames

### Including Starting Verb Extractor Function

This function is inlcuded because it has the necessary fit and Transform Functions for Tfidf and Countvectorizer Functions. If these functions are not included then there would be "No supported conversion types for dtypes error. More information can be found out here https://stackoverflow.com/questions/48467669/tfidf-transformer-sklearn-results-in-no-supported-conversion-for-types-dt

In [5]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(word_tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0][0], pos_tags[0][1]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    tokens_wihtout_sw = [w for w in tokens if w not in stopwords.words("english") ]
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    
    for tok in tokens_wihtout_sw:
        try:
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            clean_tokens.append(clean_tok)
        except:
            pass
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('vect', CountVectorizer(tokenizer=tokenize)), 
            ('tfidf', TfidfTransformer())
        ])),
        ('mclf', MultiOutputClassifier(RandomForestClassifier()))
        #('clf', RandomForestClassifier())
    ])
    
    return pipeline

In [8]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
        
        ('mclf', RandomForestClassifier())
        #('mclf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [12]:
def train():
    
    X, y, column_names = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

    model = model_pipeline()

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    display_results(y_test, y_pred)

In [13]:
train()

TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
#def display_results():
    
y_test, predicted, column_names = train()
#X, y = load_data()
#colnames = X.columns
    
#for col in colnames:
print(classification_report(y_test, predicted, target_names = column_names) )

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.